In [ ]:
from tqdm import tqdm
from queue import Queue
import cv2 as cv
import inaccel.coral as inaccel
import io
import ipywidgets as widgets
import numpy as np
import PIL.Image
import threading
import time

### Create ndarrays for the constant parameters of the accelerator
to help the scheduling we create one copy of each array for each request

In [ ]:
req_num = 4

cameraMA_l = []
cameraMA_r = []
distC_l = []
distC_r = []
irA_l = []
irA_r = []
bm_state_arr = []

with inaccel.allocator:
    for j in range(req_num):
        cameraMA_l.append(np.array([933.173, 0.0, 663.451, 0.0, 933.173, 377.015, 0.0, 0.0, 1.0], dtype = np.float32))
        cameraMA_r.append(np.array([933.467, 0.0, 678.297, 0.0, 933.467, 359.623, 0.0, 0.0, 1.0], dtype = np.float32))

        distC_l.append(np.array([-0.169398, 0.0227329, 0.0, 0.0, 0.0], dtype = np.float32))
        distC_r.append(np.array([-0.170581, 0.0249444, 0.0, 0.0, 0.0], dtype = np.float32))

        irA_l.append(np.array([0.0011976323, -0.0000000019, -0.8153011732, 0.0000000007, 0.0011976994, \
                               -0.4422348617,  0.0000126839,  0.0000001064, 0.9913820905], dtype = np.float32))
        irA_r.append(np.array([0.0011976994,  0.0000000000, -0.8047567905, -0.0000000000, 0.0011976994, \
                               -0.4420566166, -0.0000000000, -0.0000001064,  1.0000392898], dtype = np.float32))

        bm_state_arr.append(np.array([0, 15, 31, 15, 0, 48, 20, 15, 16, 3, 0], dtype = np.int32))

In [ ]:
def submitter():
    for i in range(frame_num):
        # try to read a new frame
        ret, frame = input_stream.read()
        if (ret == False):
            queue.put((None, None, None, None, None))
            break
            
        # split the frame in two, one for the left and one for the right image
        frame_left = cv.cvtColor(frame[:height, :, :], cv.COLOR_BGR2GRAY)
        frame_right = cv.cvtColor(frame[height:, :, :], cv.COLOR_BGR2GRAY)

        with inaccel.allocator:
            # convert the two frames to inaccel ndarrays and allocate the output array
            left_mat = np.array(frame_left)
            right_mat = np.array(frame_right)
            disp_mat = np.ndarray((height, width), dtype = np.uint16)

        # describe a new request
        req = inaccel.request('com.xilinx.vitis.vision.stereoBM')
        req.arg(left_mat)
        req.arg(right_mat)
        req.arg(disp_mat)
        req.arg(cameraMA_l[i % req_num])
        req.arg(cameraMA_r[i % req_num])
        req.arg(distC_l[i % req_num])
        req.arg(distC_r[i % req_num])
        req.arg(irA_l[i % req_num])
        req.arg(irA_r[i % req_num])
        req.arg(bm_state_arr[i % req_num])
        req.arg(np.int32(height))
        req.arg(np.int32(width))
        
        # try to submit that request
        try:
            future = inaccel.submit(req)
        except RuntimeError:
            print("Cannot connect to the Docker daemon. Is InAccel Coral running?")

            queue.put((None, None, None, None, None))
            break
        
        # wait while the queue is full
        while queue.full():
            time.sleep(0.01)
            
        # push the future reference as well as the ndarrays to the queue
        queue.put((future, left_mat, right_mat, disp_mat, frame))
        
        # update the queue status
        queue_status.value = queue.qsize() / queue_size

In [ ]:
def waiter():
    for i in tqdm(range(frame_num)):
        start = time.time() * 1000
        
        # try to pop from the queue
        future, left_mat, right_mat, disp_mat, frame = queue.get()
        if (future == None):
            break

        # convert the array to an image and write it to the widget
        if (print_input):
            frame = cv.resize(frame[:height, :, :], (int(width / scale_down), int(height / scale_down)))
            frame_img = PIL.Image.fromarray(frame)
            f = io.BytesIO()
            frame_img.save(f, format = 'jpeg')
            image_in.value = f.getvalue()
            image_in.width = int(width / scale_down)
            image_in.height = int(height / scale_down)
        
        # wait for the future to complete
        future.result()
        
        if (print_output):
            # normalize the output array
            disp_img = disp_mat.view(np.ndarray).astype(np.uint8)
            disp_img = cv.resize(cv.cvtColor(disp_img, cv.COLOR_GRAY2BGR), (int(width / scale_down), int(height / scale_down)))

            # convert the array to an image and write it to the widget
            disp_img = PIL.Image.fromarray(disp_img)
            f = io.BytesIO()
            disp_img.save(f, format = 'jpeg')
            image_out.value = f.getvalue()
            image_out.width = int(width / scale_down)
            image_out.height = int(height / scale_down)        
        
        # explicitly delete the inaccel ndarrays
        # this step is not necessary, as the will be garbage collected
        # but it helps free earlier the corresponding device memory
        del left_mat
        del right_mat
        del disp_mat
        
        # if sync_output is true, sleep until it's time for the next frame
        if (sync_output):
            while ((time.time() * 1000) - start < millis_per_frame):
                time.sleep(0.000001)
        
        # update the queue status
        queue.task_done()

### settings

In [ ]:
input_video = 'data/bbb_sunflower_1080p_30fps_stereo_abl.mp4'
queue_size = 16 # not necessary, but it helps with resource management
sync_output = False # force output framerate to be close to the input framerate
print_input = True # very computationally expensive, set scale_down to > 4
print_output = True # very computationally expensive, set scale_down to > 4
scale_down = 8

In [ ]:
# open the input file
input_stream = cv.VideoCapture(input_video)
if (input_stream.isOpened() == False): 
      print("Error while opening the input video stream")

# get the input's properties
frame_num = int(input_stream.get(cv.CAP_PROP_FRAME_COUNT))
height = int(input_stream.get(cv.CAP_PROP_FRAME_HEIGHT) / 2)
width  = int(input_stream.get(cv.CAP_PROP_FRAME_WIDTH))
fps = int(input_stream.get(cv.CAP_PROP_FPS))

print("Video Info:")
print("   Frames: " + str(frame_num))
print("   Dimensions: " + str(height * 2) + "x" + str(width))
print("   Framerate: " + str(fps) + " fps")

millis_per_frame = 1000 / fps

# initialize the widgets
image_in = widgets.Image()
image_out = widgets.Image()

queue_status = widgets.FloatProgress(description = 'Queue:', value = 0.0, min = 0.0, max = 1.0)

if print_input:
    images = [image_in, image_out]
else:
    images = [image_out]

image_box = widgets.HBox(images)

# initialize the queue
queue = Queue(maxsize = queue_size)

# initialize the two threads
thread1 = threading.Thread(target = waiter)
thread2 = threading.Thread(target = submitter)

# start displaying the widgets
display(queue_status)
display(image_box)

# start the threads
thread1.start()
thread2.start()

# wait for the threads to complete
queue.join()
thread1.join()
thread2.join()